# This will be the demo integration of Colivara with Langchain

#### First we need to install the dependancies

In [1]:
%pip install colivara-py --quiet
%pip install langchain --quiet
%pip install langchain-core --quiet
%pip install langchain-openai --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Import the required libraries


In [2]:
# utilities
import base64 # for converting docs binaries to base64
from pathlib import Path 
import getpass
import os 
import requests 

from colivara_py import ColiVara 

# langchain tools
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


In [3]:
# Setup Environment Variables
os.environ['OPENAI_API_KEY'] = getpass.getpass()
os.environ['COLIVARA_API_KEY'] = getpass.getpass()


#### Download example docs for our demo (you can use your own docs as well)

In [4]:
def download_file(url, local_filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Ensure the 'docs' directory exists
        os.makedirs("docs", exist_ok=True)

        # Write the content to a local file
        with open(local_filename, "wb") as f:
            f.write(response.content)
        print(f"Successfully downloaded: {local_filename}")
    else:
        print(f"Failed to download: {url}")


# URLs and local filenames
files = [
    {
        "url": "https://github.com/tjmlabs/colivara-demo/raw/main/docs/Work-From-Home%20Guidance.pdf",
        "filename": "docs/Work-From-Home-Guidance.pdf",
    },
    {
        "url": "https://github.com/tjmlabs/colivara-demo/raw/main/docs/StaffVendorPolicy-Jan2019.pdf",
        "filename": "docs/StaffVendorPolicy-Jan2019.pdf",
    },
]

In [5]:
# Download each file
for file in files:
    download_file(file["url"], file["filename"])


Successfully downloaded: docs/Work-From-Home-Guidance.pdf
Successfully downloaded: docs/StaffVendorPolicy-Jan2019.pdf


#### Start Colivara Client and Create Collection `to upsert documents into`.

In [6]:
rag_client = ColiVara(
    base_url="https://api.colivara.com", api_key=os.environ["COLIVARA_API_KEY"]
)

In [7]:
rag_client.create_collection("langchain-demo")

CollectionOut(id=78, name='langchain-demo', metadata={}, num_documents=0)

#### Upsert the documents to the collection

In [8]:
def sync_documents():
    # get all the documents under docs/ folder and upsert them to colivara
    documents_dir = Path("docs")
    files = list(documents_dir.glob("**/*"))

    for file in files:
        with open(file, "rb") as f:
            file_content = f.read()
            encoded_content = base64.b64encode(file_content).decode("utf-8")
            rag_client.upsert_document(
                name=file.name,
                document_base64=encoded_content,
                collection_name="langchain-demo",
                wait=True,
            )
            print(f"Upserted: {file.name}")

Internet connection has a great impact on this step.

In [9]:
sync_documents()

Upserted: Work-From-Home-Guidance.pdf
Upserted: StaffVendorPolicy-Jan2019.pdf


There is a step called query transformation/translation, which is the process of converting the user query into a format that the RAG model can understand. 
This step is necessary because the RAG model expects the input to be in a specific format.

***for the purpose of the demo, we will skip it and use the RAG query as the input to the RAG directly.***

Setup ended successfully.
Let's begin work.


In [10]:
query = "What is the work from home policy?"

you can do search by just using this code snippet:

```python
rag_client.search(query="some query",collection_name="collection_name", top_k=5)
``` 

In [ ]:
## you can try performing search to see results object structure

# results = rag_client.search(query, collection_name="langchain-demo")
# print(results.results)

In [11]:
# get context using the RAG
def get_context(query):
    results = rag_client.search(query=query, collection_name="langchain-demo", top_k=3)
    results = results.results

    context = []
    for result in results:
        document_title = result.document_name
        page_num = result.page_number
        base64 = result.img_base64
        # base64 doesn't have data: part so we need to add it
        if "data:image" not in base64:
            base64 = f"data:image/png;base64,{base64}"
        context.append(
            {
                "metadata": f"{document_title} - Page {page_num}",
                "base64": base64,
            }
        )
    return context

In [12]:
context = get_context(query)

Now build the prompt using the context and query, and initialize the chat model.


In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that answers questions based on the provided images/docs.",
        ),
        (
            "user",
            [
                {
                    "type": "text",
                    "text": "Here are some images for context:",
                },
                *[ 
                    {
                        "type": "image_url",
                        "image_url": {"url": image_data["base64"]},
                    }
                    for image_data in context
                ], # we are using * above to unpack the list of images (top_k=3)
                {"type": "text", "text": "Now, please answer the following question:"},
                {
                    "type": "text",
                    "text": "{query}",  # Placeholder for the user's query
                },
            ],
        ),
    ]
)

# Initialize the model
model = ChatOpenAI(model="gpt-4o", temperature=0)

In [14]:
# use chain (bro, it is called langchain for a reason) and just invoke.
chain = prompt | model
response = chain.invoke({"query": query})

In [15]:
print(response.content)

The Mount Sinai Health System's remote work policy includes the following key points:

1. **Eligibility**: Remote work is an option for employees in good standing, in positions deemed appropriate by Mount Sinai, based on job nature and institutional objectives.

2. **Approval**: Remote work requires departmental approval and periodic review. A Remote Work Agreement with specific goals and objectives is necessary.

3. **Limitations**: Remote work is not guaranteed for any job or employee and must meet business needs without discrimination.

4. **Location Requirement**: Employees working full-time remotely must spend at least one scheduled workday per year at a Mount Sinai location in New York State for tax compliance.

5. **International Work**: Remote work from outside the United States is generally prohibited, with exceptions requiring approval from the Chief Human Resources Officer.

6. **Schedule and Equipment**: The remote work schedule and any equipment provided by Mount Sinai mus

## That's it for now. You can continue to explore more features and capabilities of langchain.